In [18]:
from sentence_transformers import SentenceTransformer, models, InputExample, util
from torch.utils.data import DataLoader
import torch

In [2]:
model = SentenceTransformer("neuralmind/bert-base-portuguese-cased")

# 1. Fine-tuning

## 1.1 Preparação dos dados

In [11]:
import pandas as pd

arvore_proposicoes = pd.read_csv("../data/arvore-proposicoes.csv", encoding="utf-8")
df_proposicoes = pd.read_csv("../data/proposicao-tema-completo-sem-duplicado.csv", encoding="utf-8")
df_assunto = pd.read_csv("../data/pesquisas.csv", encoding="utf-8", delimiter=";")

In [5]:
arvore_proposicoes

,numeroSequencia,nivel,codProposicao,codProposicaoReferenciada,codProposicaoRaiz,TipoReferencia
0,8,1,13033,13335,13335,Apensação
1,7,1,13227,24325,24325,Apensação
2,23,1,13240,24480,24480,Apensação
3,24,1,13270,24480,24480,Apensação
4,3,1,13322,164372,164372,Apensação
...,...,...,...,...,...,...
344601,4,1,2216834,1672171,1672171,Proposição Acessória
344602,5,1,2216872,796203,796203,Proposição Acessória
344603,6,1,2216873,796203,796203,Proposição Acessória
344604,8,1,2216874,1344458,1344458,Proposição Acessória


In [24]:
df_proposicoes.dropna(inplace=True)
df_proposicoes.head()

,codProposicao,txtNome,txtEmenta,txtExplicacaoEmenta,txtIndexacao,imgArquivoTeorPDF,idTema
0,16357,PL 1165/1999,"Altera dispositivo da Lei nº 8.987, de 13 de f...",Estabelece que as concessionárias disponibiliz...,"Alteração, Lei das Concessões de Serviços Públ...",Ofício nº 1416 (SF) ...,34
1,20464,PL 3927/2000,Altera a composição dos Tribunais Regionais do...,"Altera a composição do TRT da 5ª região, 6ª re...","_Alteração, Lei Federal, composição, (TRT), ex...","COMISSÃO DE TRABALHO, DE ADMINISTRAÇÃO E SERVI...",34
2,20683,PL 4117/1998,Dispõe sobre o acesso a ambientes de uso colet...,...,"Autorização, pessoa portadora de deficiência, ...",Câmara dos Deputados\r\n Departament...,44
3,20857,PL 4395/1998,Estabelece as Diretrizes Nacionais de Defesa C...,...,"Fixação, diretrizes, defesa civil, (Sindec), d...",- 1 - \r\nCOMISSÃO DE CONSTITUIÇÃO E JUSTIÇA ...,41
4,26177,PL 4080/2001,Torna obrigatória a destinação de área para o ...,Obriga a destinação de área para estacionament...,"Obrigatoriedade, destinação, área, exclusivida...",\r\n \r\n807632679711411411145701111141161014...,57


In [15]:
df_assunto

,NÚMERO-PROPOSIÇÃOSILEG,TxtAssunto
0,PEC 151/2019,Solicito a consultoria elaboração de Projeto d...
1,PL 1036/2019,"tese Conamat (arts. 545, 582 e 602 da CLT ..."
2,PL 1037/2019,art. 394-A da CLT - insalubridade trabalhadora...
3,PL 1075/2019,"Solicito a analise do Projeto de Lei, anexo, p..."
4,PL 1076/2019,Elaboração de Lei Federal que obrigue todos os...
...,...,...
290,PL 914/2019,Preparar Projeto de Lei para tratar da destina...
291,PL 927/2019,Obriga as instituições e comércios a expor os...
292,PL 947/2019,SOLICITO A FEITURA DE UM PROJETO DE LEI QUE PE...
293,PL 993/2019,"Solicito reelaboração do PL 10235/2018, para r..."


In [8]:
edges = arvore_proposicoes[["codProposicao", "codProposicaoReferenciada"]].to_numpy()

array([[  13033,   13335],
       [  13227,   24325],
       [  13240,   24480],
       ...,
       [2216873,  796203],
       [2216874, 1344458],
       [2216904,  601668]])

# 2. Teste

## 2.1 Sem tuning

In [ ]:
corpus = df_proposicoes["txtEmenta"].to_numpy()
corpus_embeddings = model.encode(corpus, convert_to_tensor=True)

queries = df_assunto["TxtAssunto"].to_numpy()

In [20]:
top_k = 10
res_vector = list()
for query in queries:
    query_embedding = model.encode(query, convert_to_tensor=True)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)
    print(top_results)
    break

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


NameError: name 'corpus_embeddings' is not defined